This notebook showcases an agent designed to interact with a sql databases. The agent builds off of SQLDatabaseChain and is designed to answer more general questions about a database, as well as recover from errors.

In [1]:
def snake_to_camel(word):
    parts = word.split('_')
    return parts[0] + ''.join(x.capitalize() for x in parts[1:])

def convert_docstring(docstring):
    sentences = docstring.split("'")
    modified_sentences = []

    for sentence in sentences:
        words = sentence.split()
        modified_words = []

        for word in words:
            if '_' in word and word.islower():
                modified_words.append(snake_to_camel(word))
            else:
                modified_words.append(word)

        modified_sentence = ' '.join(modified_words)
        modified_sentences.append(modified_sentence)

    modified_docstring = "'".join(modified_sentences)
    return modified_docstring

docstring = "some_function\n\nshort_arg very_long_argument\n\nvery_long_argument and CONSTANT_VALUE\n\n'some function is converted into\n\n'someFunction`\n\nshort_arg very_long_argument different words and is converted into veryLongArgument consist of two shortArg\n\no very_long_ardment is converted into veryLongArgument\n\nо CONSTANT_VALUE' remains the same, since it is written in upper case."

modified_docstring = convert_docstring(docstring)
print(modified_docstring)



complexity worse . lengthwithin the execution limit. There are four quoted expressions: someFunction shortArg veryLongArgument veryLongArgument and CONSTANT_VALUE'some function is converted into'someFunction` shortArg veryLongArgument different words and is converted into veryLongArgument consist of two shortArg o veryLongArdment is converted into veryLongArgument о CONSTANT_VALUE'remains the same, since it is written in upper case.


In [13]:
def snake_to_camel(word):
    parts = word.split('_')
    print(parts)
    return parts[0] + ''.join(x.capitalize() for x in parts[1:])

In [14]:
snake_to_camel("very_long_argument")

['very', 'long', 'argument']


'veryLongArgument'

In [12]:
l = ["some_function", "short_arg very_long_argument","CONSTANT_VALUE"]
k= []
for ele in l:
  if " " in ele:
    ele= ele.split()
    for j in ele:
      k.append(snake_to_camel(j))
  else:
    if ele==ele.upper():
      k.append(ele)
    else:
      k.append(snake_to_camel(ele))

print(k)

['someFunction', 'shortArg', 'veryLongArgument', 'CONSTANT_VALUE']


In [ ]:
import sqlite3

# Connect to a new database file called "mydatabase.db"
conn = sqlite3.connect('Chinook.db')

# Close the database connection
conn.close()


In [ ]:
import sqlite3

# Connect to a new database file
conn = sqlite3.connect('Chinook.db')

# Open the SQL script file and read its contents
with open('./Chinook_Sqlite.sql', 'r') as f:
    sql_script = f.read()

# Execute the SQL script in the new database
conn.executescript(sql_script)

# Close the database connection
conn.close()

In [ ]:
import sqlite3

# Connect to an existing database file
conn = sqlite3.connect('Chinook.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query
cursor.execute('SELECT * FROM Artist LIMIT 10')

# Fetch the query results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and database connection
cursor.close()
conn.close()


(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "...."


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [ ]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should look at the schema of the playlisttrack table
Action: schema_sql_db
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.

> Finished chain.


'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

# Example: describing a table, recovering from an error

In [ ]:
agent_executor.run("Describe the playlistsong table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should look at the schema of the PlaylistSong table
Action: schema_sql_db
Action Input: "PlaylistSong"
Observation: Error: table_names {'PlaylistSong'} not found in database
Thought: I should look at the schema of the PlaylistTrack table
Action: schema_sql_db
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers

'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

# Example: running queries

In [ ]:
agent_executor.run("List the total sales per country. Which country's customers spent the most?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should check the schema of the relevant tables.
Action: schema_sql_db
Action Input: "Invoice, Customer"
Observation: 
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasile

c:\Users\mohammadg\Anacondanew\Anaconda3\lib\site-packages\langchain\sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


 I should query the database to get the total sales per country.
Action: query_sql_db
Action Input: SELECT Customer.Country, SUM(Invoice.Total) AS TotalSales FROM Customer INNER JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId GROUP BY Customer.Country ORDER BY TotalSales DESC LIMIT 10
Observation: [('USA', 523.0600000000003), ('Canada', 303.9599999999999), ('France', 195.09999999999994), ('Brazil', 190.09999999999997), ('Germany', 156.48), ('United Kingdom', 112.85999999999999), ('Czech Republic', 90.24000000000001), ('Portugal', 77.23999999999998), ('India', 75.25999999999999), ('Chile', 46.62)]
Thought: I now know the final answer
Final Answer: The customers from the USA spent the most, with a total of $523.06.

> Finished chain.


'The customers from the USA spent the most, with a total of $523.06.'

In [ ]:
agent_executor.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should look at the schema of the Playlist and PlaylistTrack tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Playlist, PlaylistTrack"
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I should use the Playlist and PlaylistTrack tables to get the total num

"The total number of tracks in each playlist is: '90’s Music' (1477), 'Brazilian Music' (39), 'Classical' (75), 'Classical 101 - Deep Cuts' (25), 'Classical 101 - Next Steps' (25), 'Classical 101 - The Basics' (25), 'Grunge' (15), 'Heavy Metal Classic' (26), 'Music' (6580), 'Music Videos' (1), 'On-The-Go 1' (1), 'TV Shows' (426)."

# Recovering from an error
In this example, the agent is able to recover from an error after initially trying to access an attribute (Track.ArtistId) which doesn’t exist.

In [ ]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should check the schema of the Artist and Invoice tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Artist, Invoice"
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice tab

'The top 3 best selling artists are Iron Maiden, U2, and Metallica.'

In [ ]:
agent_executor("Which Artist recorded the longest track?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Artist, Customer, Invoice, Genre, Album, InvoiceLine, MediaType, Track, Employee, Playlist, PlaylistTrack
Thought: I should check the schema of the Track table to see what columns I can query.
Action: schema_sql_db
Action Input: "Track"
Observation: 
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Ang

c:\Users\mohammadg\Anacondanew\Anaconda3\lib\site-packages\langchain\sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


 I should query the Track table for the longest track.
Action: query_sql_db
Action Input: SELECT Name, Milliseconds FROM Track ORDER BY Milliseconds DESC LIMIT 10
Observation: [('Occupation / Precipice', 5286953), ('Through a Looking Glass', 5088838), ('Greetings from Earth, Pt. 1', 2960293), ('The Man With Nine Lives', 2956998), ('Battlestar Galactica, Pt. 2', 2956081), ('Battlestar Galactica, Pt. 1', 2952702), ('Murder On the Rising Star', 2935894), ('Battlestar Galactica, Pt. 3', 2927802), ('Take the Celestra', 2927677), ('Fire In Space', 2926593)]
Thought: I now know the final answer.
Final Answer: The longest track is Occupation / Precipice with a length of 5286953 milliseconds.

> Finished chain.


{'input': 'Which Artist recorded the longest track?',
 'output': 'The longest track is Occupation / Precipice with a length of 5286953 milliseconds.'}

In [ ]:
agent_executor("plot the trend in length of tracks and give me the python code of it")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Invoice, MediaType, PlaylistTrack, Album, Genre, InvoiceLine, Artist, Playlist, Customer, Track, Employee
Thought: I should look at the schema of the Track table to see what columns I can use.
Action: schema_sql_db
Action Input: "Track"
Observation: 
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Ang

{'input': 'plot the trend length of tracks and give me the python code of it',
 'output': "The trend length of tracks can be plotted using the following python code:\n\nimport matplotlib.pyplot as plt\n\nmilliseconds = [1071, 4884, 6373, 6635, 7941, 11650, 21211, 29048, 32287, 33149]\n\nplt.plot(milliseconds)\nplt.title('Trend Length of Tracks')\nplt.xlabel('Track Number')\nplt.ylabel('Milliseconds')\nplt.show()"}